In [15]:
!pip install googlemaps folium shapely pandas


In [16]:
import googlemaps
from shapely.geometry import Point, LineString
import folium
from googlemaps.convert import decode_polyline
import pandas as pd


In [17]:
gmaps = googlemaps.Client(key='replace_client_api_key')


In [18]:
geelong = "Geelong, VIC, Australia"
melbourne = "Melbourne, VIC, Australia"

# Request directions via driving
directions_result = gmaps.directions(geelong, melbourne, mode="driving")

# Extract the polyline (route) from the response
polyline = directions_result[0]['overview_polyline']['points']


In [19]:
route_coords = decode_polyline(polyline)
route_line = LineString([(point['lat'], point['lng']) for point in route_coords])


In [20]:
buffer_distance = 0.01  # Adjust based on the desired buffer size
buffered_route = route_line.buffer(buffer_distance)


In [21]:
ev_charging_stations = []

for coord in route_coords:
    nearby_stations = gmaps.places_nearby(location=(coord['lat'], coord['lng']),
                                          radius=buffer_distance * 100000,  # Convert to meters
                                          type='charging_station')
    ev_charging_stations.extend(nearby_stations['results'])

# Remove duplicates by place_id
ev_charging_stations = {station['place_id']: station for station in ev_charging_stations}.values()


In [23]:
gmaps = googlemaps.Client(key='replace_client_api_key')

# Simple geocoding request to test the API key
geocode_result = gmaps.geocode('Melbourne, VIC, Australia')

print(geocode_result)


[{'address_components': [{'long_name': 'Melbourne', 'short_name': 'Melbourne', 'types': ['colloquial_area', 'locality', 'political']}, {'long_name': 'Victoria', 'short_name': 'VIC', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Australia', 'short_name': 'AU', 'types': ['country', 'political']}], 'formatted_address': 'Melbourne VIC, Australia', 'geometry': {'bounds': {'northeast': {'lat': -37.5112737, 'lng': 145.5125288}, 'southwest': {'lat': -38.4338593, 'lng': 144.5937418}}, 'location': {'lat': -37.8136276, 'lng': 144.9630576}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': -37.5112737, 'lng': 145.5125288}, 'southwest': {'lat': -38.4338593, 'lng': 144.5937418}}}, 'place_id': 'ChIJ90260rVG1moRkM2MIXVWBAQ', 'types': ['colloquial_area', 'locality', 'political']}]


In [24]:
# Testing places_nearby with a fixed location and radius
test_location = (-37.8136276, 144.9630576)  # Melbourne's coordinates
test_radius = 5000  # 5 km

try:
    nearby_stations = gmaps.places_nearby(location=test_location,
                                          radius=test_radius,
                                          type='charging_station')
    print(nearby_stations)
except googlemaps.exceptions.ApiError as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"Unexpected Error: {e}")


{'html_attributions': [], 'next_page_token': 'AelY_CvAxUMK_LPbix_roHul1MIPdeh_pdQEFZ7Rp0fJy9gpDjv-6vb81BUY-nLZnvhB_8h8GTi6uik2fsYHicr409N32CngdTXcdUSjmFS0ig7BrqkL10gKa-DRIPidXkIfFq75Y67tIo2D-qQwdCDfpQL-Y92goEEfkEGp0UGUn8D16QEYuljZkoDEFwJn_NZLmytNexLVq1fSrJEI0q4qIY0ohgkzhLAwMA1J4xCWofQDvWAr89OwrAeK78z3nOrT5c2dnJ-VmO9c5W7Dq1hyhMv9E16R9-wy-6oSgar7o1GgRQuIA5Z_mz04vIXIu-M766FMahTzI_aJSgyFJcEL3TyTUfDvOuhs4vQu6xbzHF78KfJAMx9dTy81zf8OETJK7cO7A9JDAiMbN1eDeIff6oq6SVVWGIiXGp_X787EyU70aLjbq-xzYKErNokjsPZti2qa', 'results': [{'geometry': {'location': {'lat': -37.8136276, 'lng': 144.9630576}, 'viewport': {'northeast': {'lat': -37.5112736581378, 'lng': 145.512528780816}, 'southwest': {'lat': -38.43385934606901, 'lng': 144.5937417793496}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Melbourne', 'photos': [{'height': 2160, 'h

In [25]:
try:
    nearby_stations = gmaps.places_nearby(location=test_location, radius=test_radius)
    print(nearby_stations)
except googlemaps.exceptions.ApiError as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"Unexpected Error: {e}")


{'html_attributions': [], 'next_page_token': 'AelY_Cv2dEiPn3zCNjCL2P-_2P5jzVUNmFGvk1j6yN22hItXmfUglNXrpSSVTe03yTqb47fBGEq7M4Wt0rcmUqn82JsSp5oBlypdlh9qKVceOJ4U2NXLRe1qq1d2qm_oZyO8KPhuTsWYByWT4_D9K-UOPpUah8omoLmlR7bMEZ_dCesoRsIDxTwskp8ugLaOe6h0lI4VSdFEDNVYV2F65BeVl-Gzs5T9Q9QHsafq5McAK0Vq8xFruu-aHKsc23G46MXLSTpBAT5SMuMcCn5VCoRYEN4rvraHNnVEvfHBHT_fDbbQnppHgjq2_u4ZyUGwSUlkitbjDlRxozKpgqyPvB7BqmxLURwjuRAbIPzGWyrDc9ICIa15AkIbUyCrQDdKwlP38XKisiv61b2-2T6AhyxNK7S4sCvunCacXxSTcxwyK-Vn', 'results': [{'geometry': {'location': {'lat': -37.8136276, 'lng': 144.9630576}, 'viewport': {'northeast': {'lat': -37.5112736581378, 'lng': 145.512528780816}, 'southwest': {'lat': -38.43385934606901, 'lng': 144.5937417793496}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Melbourne', 'photos': [{'height': 2160, 'html_attributions': ['<a 

In [26]:
try:
    nearby_stations = gmaps.places_nearby(location=test_location, radius=test_radius, keyword='EV charging')
    print(nearby_stations)
except googlemaps.exceptions.ApiError as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"Unexpected Error: {e}")


{'html_attributions': [], 'next_page_token': 'AelY_CuU_uNJV8_kD0TSOG9qlMFtyXGhZ5pInJqHRSPUhagVeLa3ZGoojqrp7VwnnPHH_qW9ByOyPRuanD5AYi5iJNLTHZhcuo6TCwQV1Z-Y611lR5IuUCrZfPV0FsCsrBFVq2d_qY-_mDdSeuOyZogw9-n-Dg4EaIophN38f-Awdwp1OGhuWbV1krNIMf7_LagD-buxx978SEmaAWaZSkHaFSAxkwu97n3mCEoiLH7fRAHcI7tmE8bicZnuOxqWZ4idFfs0XWe1Pc229Fk3i2NARCnTI9sh1ZQ4_5LBGjzvEJqTcbkeC8GtLSi9yVMtDb_grEsli_GDDsHCmmJYUmv0e_pvQv6mijA6JLQbR9I7FpbPbNnGmyuVF9WO1NQkHy6XNDH1AdI6b8NfP2t5SkNUzAOuvYLfgPk5qbk9JPv5zEjHnBfxCMqShYtJYrzgfw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -37.8044708, 'lng': 144.9932696}, 'viewport': {'northeast': {'lat': -37.80312432010728, 'lng': 144.9946531798927}, 'southwest': {'lat': -37.80582397989272, 'lng': 144.9919535201073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Evi

In [27]:
# If there are more results, retrieve them using the next_page_token
if 'next_page_token' in nearby_stations:
    next_page_token = nearby_stations['next_page_token']
    try:
        more_stations = gmaps.places_nearby(location=test_location,
                                            radius=test_radius,
                                            keyword='EV charging',
                                            page_token=next_page_token)
        print(more_stations)
    except googlemaps.exceptions.ApiError as e:
        print(f"API Error: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")


{'html_attributions': [], 'next_page_token': 'AelY_CujcnztsKVphc-w36PMINPByk-YcMaCRxLwBy3rUKdEfLuIxDQLl953K5Cb1i7u8jQMLGqq-xLTjD2H4PD-ZYb0fNXK03VmeHE-cvmsuwxVCp71QrOLGxsx2To0AVQGlDzODnGQkChcScRnVMeDs5eq4YmkX7hv4kLEU8-U8aFT3j3LkbLKkz_ymENxav2f2OHOTZZMhEMJq9Vm518eIJl1_MJt_CirqZ5KF1Cjh61EQUjDYJaKLoXhwHCI7trJ4vSSgVvV9OrQYonCAY44c80z1OrPQW0qutTLVKAHMnuNWLn-N8gwxCypSDW3q-LAOhnOnmI2E0rccKccd2jqUdnTvDH6Gfuo1Hor-VG5MsZj_4ZQP9kVcvduKShXS7kXZf0VVAOXztrYJpuGP_hQdeGNIuyPRn5J3n56iKBjSYZXitXheaBVRBoGRuHXbm4jIudn0Sw8dZWCieeJsRoefb1n7xkXNNkhy8Wl0JmLPgofvNAp08WCQuE4ol8icOiZoVvvewJYGCdJlroXqF1LqU73EVC-mVezfI5osO2xR4KFHxdy8cIuDZSnlMkud4eHc6H63Kc0U1wiThIXNjGoo2C4tXd3VyduLkvZR_HeVt6cEQuuy_uCbTBJ0_nmEhqu3XVDZPsTZTjUY8SvsmM', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -37.8319137, 'lng': 144.9609308}, 'viewport': {'northeast': {'lat': -37.83044412010727, 'lng': 144.9622220798927}, 'southwest': {'lat': -37.83314377989272, 'lng': 144.9595224201073}}}, 'icon': 'https://maps.g

In [28]:
import folium

# Create a map centered around the test location
map_center = [test_location[0], test_location[1]]
map = folium.Map(location=map_center, zoom_start=13)

# Add markers for each charging station
for station in nearby_stations['results']:
    lat = station['geometry']['location']['lat']
    lng = station['geometry']['location']['lng']
    name = station['name']
    folium.Marker([lat, lng], popup=name).add_to(map)

# Display the map
map


In [29]:
import googlemaps
import folium
from googlemaps.convert import decode_polyline
import time

# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(key='replace_client_api_key')

# Step 1: Manually enter two locations
origin = input("Enter the starting location: ")
destination = input("Enter the destination location: ")

# Step 2: Get directions between the two locations
directions_result = gmaps.directions(origin, destination, mode="driving")

# Extract the route polyline
route_polyline = directions_result[0]['overview_polyline']['points']

# Decode the polyline to get a list of latitude/longitude pairs
route_coords = decode_polyline(route_polyline)

# Step 3: Find EV charging stations along the route
ev_charging_stations = []
buffer_distance = 5000  # 5 km radius for nearby places

for coord in route_coords:
    nearby_stations = gmaps.places_nearby(location=(coord['lat'], coord['lng']),
                                          radius=buffer_distance,
                                          keyword='EV charging')
    ev_charging_stations.extend(nearby_stations['results'])

    # Handle pagination if there are more results
    while 'next_page_token' in nearby_stations:
        time.sleep(2)  # Delay to handle API rate limiting
        nearby_stations = gmaps.places_nearby(location=(coord['lat'], coord['lng']),
                                              radius=buffer_distance,
                                              keyword='EV charging',
                                              page_token=nearby_stations['next_page_token'])
        ev_charging_stations.extend(nearby_stations['results'])

# Remove duplicates by place_id
unique_ev_charging_stations = {station['place_id']: station for station in ev_charging_stations}.values()

# Step 4: Visualize the route and EV charging stations on a map
map_center = [(route_coords[0]['lat'] + route_coords[-1]['lat']) / 2,
              (route_coords[0]['lng'] + route_coords[-1]['lng']) / 2]
map = folium.Map(location=map_center, zoom_start=10)

# Add the route to the map
folium.PolyLine([(point['lat'], point['lng']) for point in route_coords], color="blue", weight=2.5).add_to(map)

# Add EV charging stations to the map
for station in unique_ev_charging_stations:
    lat = station['geometry']['location']['lat']
    lng = station['geometry']['location']['lng']
    name = station['name']
    folium.Marker([lat, lng], popup=name).add_to(map)

# Display the map in the notebook
map


Enter the starting location: Melbourne
Enter the destination location: Sydney
